# Importaciones

In [187]:
"""Módulo para crear un modelo predictivo de precios de Airbnb en la Comunidad de Madrid."""

import pandas as pd

from sklearn.preprocessing import LabelEncoder

# Descargar los datos

In [11]:
data_location = "https://data.insideairbnb.com/spain/comunidad-de-madrid/madrid/2024-03-22/data/listings.csv.gz"

In [12]:
!wget https://data.insideairbnb.com/spain/comunidad-de-madrid/madrid/2024-03-22/data/listings.csv.gz

--2024-06-01 19:46:38--  https://data.insideairbnb.com/spain/comunidad-de-madrid/madrid/2024-03-22/data/listings.csv.gz
Resolving data.insideairbnb.com (data.insideairbnb.com)... 18.154.48.59, 18.154.48.87, 18.154.48.41, ...
Connecting to data.insideairbnb.com (data.insideairbnb.com)|18.154.48.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13297683 (13M) [application/x-gzip]
Saving to: ‘listings.csv.gz’

listings.csv.gz     100%[===================>]  12.68M  17.1MB/s    in 0.7s    

2024-06-01 19:46:39 (17.1 MB/s) - ‘listings.csv.gz’ saved [13297683/13297683]



In [344]:
# Reasoning behind which columns are removed, and which require text processing
columns = [
    'id',                     # Remove - Identifier
    'listing_url',            # Remove - URL with ID at end
    'scrape_id',              # Remove - details of scrape
    'last_scraped',           # Remove - detail of scrape
    'source',                 # Remove - detail of scrape
    'name',                   # Do text processing
    'description',            # Do text processing
    'neighborhood_overview',  # Do text processing
    'picture_url',            # Remove - URl
    'host_id',                # Could learn relation between host and price
    'host_url',               # Remove - redundant with host_id
    'host_name',              # Do text processing
    'host_since',             # Convert to number of months or years 
    'host_location',          # Do text processing
    'host_about',             # Do text processing
    'host_response_time', 'host_response_rate',
    'host_acceptance_rate', 'host_is_superhost',
    'host_thumbnail_url',     # Remove
    'host_picture_url',       # Remove
    'host_neighbourhood',     # Do text processing
    'host_listings_count', 'host_total_listings_count',
    'host_verifications',     #
    'host_has_profile_pic', 'host_identity_verified',
    'neighbourhood',          #  Remove - redundant
    'neighbourhood_cleansed', #  Do text processing
    'neighbourhood_group_cleansed',
    'latitude', 'longitude',  # Consider removing, redundant with neighbourhood_cleansed
    'property_type',          #  Do text processing
    'room_type', 'accommodates', 'bathrooms',
    'bathrooms_text',         # Remove - largely redundant
    'bedrooms', 'beds', 'amenities',
    'price',                  # Dependent variable
    'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
    'maximum_minimum_nights', 'minimum_maximum_nights',
    'maximum_maximum_nights', 'minimum_nights_avg_ntm',
    'maximum_nights_avg_ntm',
    'calendar_updated',       # Remove - 0 not null
    'has_availability',
    'availability_30', 'availability_60', 'availability_90',
    'availability_365',
    'calendar_last_scraped',  # Remove - same across all rows
    'number_of_reviews',
    'number_of_reviews_ltm', 'number_of_reviews_l30d',
    'first_review', 'last_review',  # Convert to days ago
    'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin',
    'review_scores_communication', 'review_scores_location',
    'review_scores_value',
    'license',                # Remove 
    'instant_bookable',
    'calculated_host_listings_count',
    'calculated_host_listings_count_entire_homes',
    'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms', 'reviews_per_month'   
]

data_types = {
    'host_response_time': pd.StringDtype(),
    'host_response_rate': pd.StringDtype(),
    'host_acceptance_rate': pd.StringDtype(),
    'host_is_superhost': pd.StringDtype(),
    'host_has_profile_pic': pd.StringDtype(),
    'host_identity_verified': pd.StringDtype(),
    'room_type': pd.StringDtype(),
    'has_availability': pd.StringDtype(),
    'instant_bookable': pd.StringDtype(),
    'name': pd.StringDtype(),
    'description': pd.StringDtype(),
    'neighborhood_overview': pd.StringDtype(),
    'host_name': pd.StringDtype(),
    'host_location': pd.StringDtype(),
    'host_about': pd.StringDtype(),
    'host_neighbourhood': pd.StringDtype(),
    'neighbourhood_cleansed': pd.StringDtype(),
    'neighbourhood_group_cleansed': pd.StringDtype(),
    'property_type': pd.StringDtype(),
    'host_verifications': pd.StringDtype(),
    'amenities': pd.StringDtype(),
}

# Cargar los datos y crear subconjuntos

In [387]:
parse_dates = ['host_since','first_review','last_review']

data = pd.read_csv('listings.csv.gz', parse_dates=parse_dates, dtype=data_types)

# Select only data where there exists a price
data = data[data['price'].notnull()]

y = data['price']

data.shape

(20688, 75)

In [388]:
cols_of_interest = [
    'host_id', 'host_since', 'host_response_time', 'host_response_rate',
    'host_acceptance_rate', 'host_is_superhost', 'host_listings_count',
    'host_total_listings_count', 'host_verifications', 'host_has_profile_pic',
    'host_identity_verified', 'latitude', 'longitude', 'room_type', 'accommodates',
    'bathrooms', 'bedrooms', 'beds', 'minimum_nights', 'maximum_nights',
    'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights',
    'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
    'has_availability', 'availability_30', 'availability_60', 'availability_90',
    'availability_365', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
    'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication',
    'review_scores_location', 'review_scores_value', 'instant_bookable',
    'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',
    'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms',
    'reviews_per_month'
                   ]

In [389]:
# Columns to that require text processing
cols_text = [
    'name', 'description', 'neighborhood_overview', 'host_name', 
    'host_location', 'host_about', 'host_neighbourhood', 'neighbourhood_cleansed',
    'neighbourhood_group_cleansed', 'property_type', 'amenities'
            ]

In [390]:
# Create data subsets
data_numerical = data[cols_of_interest]
data_text = data[cols_text]

data_numerical.shape, data_text.shape

((20688, 49), (20688, 11))

# Ingeniería de Características

## Cleaning numerical columns

In [391]:
# TRICKIER COLS FOR LATER
# 'amenities'            # list of strings lalala
pd.options.mode.copy_on_write = True

In [392]:
# Calculate years hosting
data_numerical['host_since_year'] = data_numerical['host_since'].apply(lambda x: x.year)
data_numerical['host_years'] = data_numerical['host_since_year'].apply(lambda x: 2024 - x)
data_numerical = data_numerical.drop(['host_since', 'host_since_year'], axis=1)

# Calculate years since first review
data_numerical['first_review_year'] = data_numerical['first_review'].apply(lambda x: x.year)
data_numerical['first_review_age'] = data_numerical['first_review_year'].apply(lambda x: 2024 - x)
data_numerical = data_numerical.drop(['first_review', 'first_review_year'], axis=1)

# Calculate years since last review
data_numerical['last_review_year'] = data_numerical['last_review'].apply(lambda x: x.year)
data_numerical['last_review_age'] = data_numerical['last_review_year'].apply(lambda x: 2024 - x)
data_numerical = data_numerical.drop(['last_review', 'last_review_year'], axis=1)

In [393]:
# Change t f columns to 1 and 0
columns_t_f = ['host_is_superhost','host_has_profile_pic', 'host_identity_verified','has_availability','instant_bookable']
t_f = {'t':'1', 'f':'0'}

for col in columns_t_f:
    data_numerical[col] = data_numerical[col].replace(t_f)

data_numerical[columns_t_f] = data_numerical[columns_t_f].apply(pd.to_numeric)

In [394]:
# Strip % and convert int
columns_percent = ['host_response_rate','host_acceptance_rate']

for col in columns_percent:
    data_numerical[col] = data_numerical[col].str.replace('%', '').apply(pd.to_numeric)

In [395]:
# Convert to category strings to numerical codes
columns_categorical = ['host_response_time', 'room_type']

label_encoder = LabelEncoder()
data_numerical[columns_categorical] = data_numerical[columns_categorical].apply(lambda series: pd.Series(
    LabelEncoder().fit_transform(series[series.notnull()]),
))

In [396]:
# Clean string
data_numerical['host_verifications'] = data_numerical['host_verifications'].str.replace(r"[\'\[\]]", "", regex=True)
data_numerical['host_verifications'] = data_numerical['host_verifications'].str.replace(" ", "")

verification_cols = ['email_verified', 'phone_verified','work_email_verified']

# Expand columns
data_numerical[verification_cols] = data_numerical['host_verifications'].str.split(',', expand=True)

# Convert to 1 and 0
data_numerical[verification_cols] = data_numerical[verification_cols].notnull().astype(int)

# Drop original column
data_numerical = data_numerical.drop(['host_verifications'], axis=1)

In [397]:
data_numerical.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20688 entries, 0 to 26023
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   host_id                                       20688 non-null  int64  
 1   host_response_time                            14276 non-null  float64
 2   host_response_rate                            19208 non-null  float64
 3   host_acceptance_rate                          19889 non-null  float64
 4   host_is_superhost                             20578 non-null  Int64  
 5   host_listings_count                           20688 non-null  float64
 6   host_total_listings_count                     20688 non-null  float64
 7   host_has_profile_pic                          20688 non-null  Int64  
 8   host_identity_verified                        20688 non-null  Int64  
 9   latitude                                      20688 non-null  floa

In [398]:
data_numerical.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
host_id,20688.0,241882664.465536,194127473.912554,17453.0,45602149.25,216203351.0,436783593.0,567968941.0
host_response_time,14276.0,2.581535,0.8088,0.0,2.0,3.0,3.0,3.0
host_response_rate,19208.0,93.427374,19.02392,0.0,98.0,100.0,100.0,100.0
host_acceptance_rate,19889.0,86.220826,25.515849,0.0,88.0,99.0,100.0,100.0
host_is_superhost,20578.0,0.271795,0.444896,0.0,0.0,0.0,1.0,1.0
host_listings_count,20688.0,41.172032,100.214239,1.0,1.0,4.0,26.0,2257.0
host_total_listings_count,20688.0,51.989946,146.979071,1.0,2.0,6.0,32.0,5669.0
host_has_profile_pic,20688.0,0.972109,0.164663,0.0,1.0,1.0,1.0,1.0
host_identity_verified,20688.0,0.982502,0.131121,0.0,1.0,1.0,1.0,1.0
latitude,20688.0,40.421627,0.023319,40.331397,40.40954,40.419973,40.431423,40.53553


In [399]:
# Handle Nulls

In [400]:
# Normalize